In [1]:
# import dependencies
import pandas as pd
import os

In [2]:
# define source data locations, import and combine data
file_1 = os.path.join('HeroesOfPymoli','purchase_data.json')
file_2 = os.path.join('HeroesOfPymoli','purchase_data2.json')
df_1 = pd.read_json(file_1)
df_2 = pd.read_json(file_2)
df = df_1.append(df_2,ignore_index=True)

In [3]:
# Report unique player count
player_count = len(df['SN'].unique())
pd.DataFrame({'Total Players':[player_count]})

,Total Players
0,612


In [4]:
# Count unique items
item_count = len(df['Item Name'].unique())
# Calculate average price and change format
average_price = '${:,.2f}'.format(df['Price'].mean())
# Count total number of purchases
purchase_count = df['Item Name'].count()
# Calculate total revenue and change format
revenue = '${:,.2f}'.format(df['Price'].sum())
# Report purchase analysis
pd.DataFrame([[item_count,average_price,purchase_count,revenue]],
             columns=['Number of Unique Items',
                      'Average Purchase Price',
                      'Total Number of Purchases',
                      'Total Revenue'])

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,180,$2.93,858,"$2,514.43"


In [5]:
# Remove duplicate players from the purchase list
unique_df = df.drop_duplicates(['SN'])
# Count unique players by gender
gender_df = pd.DataFrame(unique_df['Gender'].value_counts())
# Change column header
gender_df.rename(columns = {'Gender':'Total Count'},inplace=True)
# Calculate the total number of unique players
gender_total = gender_df['Total Count'].sum()
# Add new column to show % of total players by gender
gender_df['Percentage of Players'] = (gender_df['Total Count'] / gender_total * 100).apply('{:,.2f}'.format)
# Move columns around and report out gender demographics
gender_df = gender_df[['Percentage of Players','Total Count']]
gender_df


,Percentage of Players,Total Count
Male,80.88,495
Female,17.65,108
Other / Non-Disclosed,1.47,9


In [6]:
# Define column names
column_name = {'count':'Purchase Count','mean':'Average Purchase Price','sum':'Total Purchase Value'}
# Group by gender and count number of purchases, average price and total purchase price
purchase_df = df.groupby(['Gender']).agg({'Gender':'count','Price':['mean','sum']}).rename(columns = column_name)
# Remove level 0 of multi-index on the columns
purchase_df.columns = purchase_df.columns.droplevel(0)
# Calculate normalized totals and change the format
purchase_df['Normazlied Totals'] = (purchase_df['Total Purchase Value'] / gender_df['Total Count']).apply('${:,.2f}'.format)
# Change the format of average price
purchase_df['Average Purchase Price'] = purchase_df['Average Purchase Price'].apply('${:,.2f}'.format)
# Change the format of total purchase price
purchase_df['Total Purchase Value'] = purchase_df['Total Purchase Value'].apply('${:,.2f}'.format)
# Report out purchasing analysis by gender
purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normazlied Totals
Gender,,,,
Female,149,$2.85,$424.29,$3.93
Male,697,$2.94,"$2,052.28",$4.15
Other / Non-Disclosed,12,$3.15,$37.86,$4.21


In [7]:
# Create bins for different age groups
bins = [0,9,14,19,24,29,34,39,100]
# Define group labels
group_name = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
# Assign age to differnt age groups
age_df = pd.cut(df['Age'],bins,labels = group_name)
# Add new column to show age group
df['Age group'] = age_df
# Count number of purchase by age group
age_group_count = df['Age group'].value_counts()

In [8]:
# Group data by age group and calculate number of purchase, average price and total purchase price, rename column headers
age_df = df.groupby(['Age group']).agg({'SN':'count','Price':['mean','sum']}).rename(columns = column_name)
# Remove level 0 of multi-index on the columns
age_df.columns = age_df.columns.droplevel(0)
# Calculate normalized totals and change the format
age_df['Normalized Totals'] = (age_df['Total Purchase Value'] / age_group_count).apply('${:,.2f}'.format)
# Change the format of average price
age_df['Average Purchase Price'] = age_df['Average Purchase Price'].apply('${:,.2f}'.format)
# Change the format of total purchase price
age_df['Total Purchase Value'] = age_df['Total Purchase Value'].apply('${:,.2f}'.format)
# Report out age demographics and purhcase analysis
age_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age group,,,,
<10,33,$2.95,$97.28,$2.95
10-14,38,$2.79,$105.91,$2.79
15-19,144,$2.89,$416.83,$2.89
20-24,372,$2.92,"$1,087.66",$2.92
25-29,134,$2.96,$396.44,$2.96
30-34,71,$2.97,$211.14,$2.97
35-39,48,$2.93,$140.77,$2.93
40+,18,$3.24,$58.40,$3.24


In [9]:
# Group data by player and calcuate the number of purchases, average price and total purchase value
top_spender_df = df.groupby(['SN']).agg({'SN':'count','Price':['mean','sum']}).rename(columns = column_name)
# Remove level 0 of multi-index on the columns
top_spender_df.columns = top_spender_df.columns.droplevel(0)
# Select the top 5 players
top_spender_df = top_spender_df.nlargest(5,'Total Purchase Value')
# Change the format of average price
top_spender_df['Average Purchase Price'] = top_spender_df['Average Purchase Price'].apply('${:,.2f}'.format)
# Change the format of total purchase price
top_spender_df['Total Purchase Value'] = top_spender_df['Total Purchase Value'].apply('${:,.2f}'.format)
# Report out top spender analysis
top_spender_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Aerithllora36,4,$3.77,$15.10
Saedue76,4,$3.39,$13.56
Sondim43,4,$3.25,$13.02
Mindimnya67,4,$3.18,$12.74


In [10]:
# Define column names
column_name = {'count':'Purchase Count','min':'Item Price','sum':'Total Purchase Value'}
# Group data by items and calcuate the number of purchases, item price (min price or max price) and total purchase value
popular_df = df.groupby(['Item ID','Item Name']).agg({'Item ID':'count','Price':['min','sum']}).rename(columns = column_name)
# Remove level 0 of multi-index on the columns 
popular_df.columns = popular_df.columns.droplevel(0)
# Select the top 5 sold items
popular_df = popular_df.nlargest(5,'Purchase Count')
# Change the format of item price
popular_df['Item Price'] = popular_df['Item Price'].apply('${:,.2f}'.format)
# Change the format of total purchase price
popular_df['Total Purchase Value'] = popular_df['Total Purchase Value'].apply('${:,.2f}'.format)
# Report out most popular items
popular_df

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
84,Arcane Gem,12,$2.23,$29.34
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
31,Trickster,10,$2.07,$23.22
13,Serenity,9,$1.49,$13.41
34,Retribution Axe,9,$4.14,$37.26


In [11]:
# Define column names
column_name = {'count':'Purchase Count','min':'Item Price','sum':'Total Purchase Value'}
# Group data by items and calcuate the number of purchases, item price (min price or max price) and total purchase value
profitable_df = df.groupby(['Item ID','Item Name']).agg({'Item ID':'count','Price':['min','sum']}).rename(columns = column_name)
# Remove level 0 of multi-index on the columns 
profitable_df.columns = profitable_df.columns.droplevel(0)
# Select the top 5 items by total purchase value
profitable_df = profitable_df.nlargest(5,'Total Purchase Value')
# Change the format of item price
profitable_df['Item Price'] = profitable_df['Item Price'].apply('${:,.2f}'.format)
# Change the format of total purchase price
profitable_df['Total Purchase Value'] = profitable_df['Total Purchase Value'].apply('${:,.2f}'.format)
# Report out most profitable items
popular_df

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
84,Arcane Gem,12,$2.23,$29.34
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
31,Trickster,10,$2.07,$23.22
13,Serenity,9,$1.49,$13.41
34,Retribution Axe,9,$4.14,$37.26
